# WOfS Validation_Data-Clean  <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2),
[ga_ls8c_wofs_2_summary ](https://explorer.digitalearth.africa/ga_ls8c_wofs_2_summary)

## Background
The [Water Observations from Space (WOfS)](https://www.ga.gov.au/scientific-topics/community-safety/flood/wofs/about-wofs) is a derived product from Landsat 8 satellite observations as part of provisional Landsat 8 Collection 2 surface reflectance and shows surface water detected in Africa.
Individual water classified images are called Water Observation Feature Layers (WOFLs), and are created in a 1-to-1 relationship with the input satellite data. 
Hence there is one WOFL for each satellite dataset processed for the occurrence of water.

The data in a WOFL is stored as a bit field. This is a binary number, where each digit of the number is independantly set or not based on the presence (1) or absence (0) of a particular attribute (water, cloud, cloud shadow etc). In this way, the single decimal value associated to each pixel can provide information on a variety of features of that pixel. 
For more information on the structure of WOFLs and how to interact with them, see [Water Observations from Space](../Datasets/Water_Observations_from_Space.ipynb) and [Applying WOfS bitmasking](../Frequently_used_code/Applying_WOfS_bitmasking.ipynb) notebooks. 

## Description
This notebook explains how you can compile tables from Collect Earth Online tool from each partner institution and make them analysis-ready for WOfS analysis and accuracy assessment. 

The notebook demonstrates how to:

1. Load collected validation points as a list of observations each has a location and month
2. Data wrangling including cleaning the table, and mapping each point two twelve month observation 

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

After finishing the analysis, you can modify some values in the "Analysis parameters" cell and re-run the analysis to load WOFLs for a different location or time period.

### Load packages

In [1]:
%matplotlib inline

import datacube
from datacube.utils import masking, geometry 
import sys
import os
import dask 
import rasterio, rasterio.features
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import subprocess as sp
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

sys.path.append("../Scripts")
from deafrica_plotting import display_map, rgb
from deafrica_spatialtools import xr_rasterize
from deafrica_datahandling import wofs_fuser, mostcommon_crs,load_ard
from rasterio.mask import mask

### Connect to the datacube

In [2]:
dc = datacube.Datacube()

### Analysis parameters

In [3]:
#make sure that validation points have at least three columns : location (x,y), class, as well as 12 records for each observation  
#Path to the validation data points csv file 
CEO = '../Supplementary_data/Validation/CEO_4_AFRIGIST_2020-09-15.csv'

### Loading Dataset

In [4]:
#Read in the validation data csv
df = pd.read_csv(CEO, delimiter=",")
df.columns

Index(['PLOT_ID', 'SAMPLE_ID', 'LON', 'LAT', 'FLAGGED', 'ANALYSES', 'USER_ID',
       'COLLECTION_TIME', 'ANALYSIS_DURATION', 'IMAGERY_TITLE',
       'SENTINEL2MOSAICYEARMONTH', 'GEEIMAGECOLLECTIONASSETID',
       'GEEIMAGECOLLECTIONENDDATE', 'GEEIMAGECOLLECTIONSTARTDATE', 'PL_PLOTID',
       'ENTER MONTHS[1-12] IN 2018, WATER WAS OBSERVED?',
       'ENTER MONTHS[1-12] IN 2018, WATER WAS NOT OBSERVED?',
       'ENTER MONTHS[1-12] IN 2018, IMAGE WAS BAD?',
       'ENTER MONTHS[1-12] IN 2018, THAT YOU ARE UNSURE IF YOU OBSERVE WATER OR NOT? ',
       'WHAT IS THE FEATURE?', 'COMMENT'],
      dtype='object')

In [5]:
ground_truth = df.drop(['SAMPLE_ID','USER_ID','IMAGERY_TITLE','COLLECTION_TIME','ANALYSIS_DURATION','GEEIMAGECOLLECTIONASSETID','PL_PLOTID'], axis=1)

In [6]:
ground_truth.columns

Index(['PLOT_ID', 'LON', 'LAT', 'FLAGGED', 'ANALYSES',
       'SENTINEL2MOSAICYEARMONTH', 'GEEIMAGECOLLECTIONENDDATE',
       'GEEIMAGECOLLECTIONSTARTDATE',
       'ENTER MONTHS[1-12] IN 2018, WATER WAS OBSERVED?',
       'ENTER MONTHS[1-12] IN 2018, WATER WAS NOT OBSERVED?',
       'ENTER MONTHS[1-12] IN 2018, IMAGE WAS BAD?',
       'ENTER MONTHS[1-12] IN 2018, THAT YOU ARE UNSURE IF YOU OBSERVE WATER OR NOT? ',
       'WHAT IS THE FEATURE?', 'COMMENT'],
      dtype='object')

In [7]:
ground_truth.shape

(172, 14)

In [8]:
ground_truth = ground_truth.rename(columns={'WHAT IS THE FEATURE?':'CLASS','ENTER MONTHS[1-12] IN 2018, WATER WAS OBSERVED?':'WATER','SENTINEL2MOSAICYEARMONTH':'S2DATE',
                                            'ENTER MONTHS[1-12] IN 2018, WATER WAS NOT OBSERVED?':'NO_WATER','ENTER MONTHS[1-12] IN 2018, IMAGE WAS BAD?':'BAD_IMAGE',
                                             'ENTER MONTHS[1-12] IN 2018, THAT YOU ARE UNSURE IF YOU OBSERVE WATER OR NOT? ':'NOT_SURE',
                                            'GEEIMAGECOLLECTIONENDDATE':'ENDDATE','GEEIMAGECOLLECTIONSTARTDATE':'STARTDATE'})

In [9]:
ground_truth

,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,S2DATE,ENDDATE,STARTDATE,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT
0,137483360,-13.157104,8.894824,False,1,2018 - 2018,NaN,NaN,1-Dec,0,6-Sep,0,Open water - marine,NaN
1,137483361,6.884502,8.891968,False,1,2018 - 2018,NaN,NaN,1-Dec,0,4-Sep,0,Open water - freshwater,NaN
2,137483362,-3.870084,8.717834,False,1,2018 - 2018,NaN,NaN,1-Dec,0,5-Sep,0,Open water - freshwater,NaN
3,137483363,-12.511312,8.694765,False,1,2018 - 2018,NaN,NaN,1-Dec,0,5-Oct,0,Open water - Constructed (e.g. aquaculture),constructed reservoir
4,137483364,-13.240743,8.657429,False,1,2018 - 2018,NaN,NaN,1-Dec,0,6-Oct,0,Open water - marine,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,137483525,11.249268,6.156226,False,1,2018 - December,NaN,NaN,"1-7,9-12",8,8,0,Open water - freshwater,NaN
168,137483526,10.573317,5.746073,False,1,2018 - December,NaN,NaN,1-Dec,0,0,0,Open water - freshwater,NaN
169,137483527,8.750363,4.708335,False,1,2018 - December,NaN,NaN,"1-2,10-12",3-Sep,3-Sep,0,Open water - freshwater,NaN
170,137483528,9.411390,4.659262,False,1,2018 - December,NaN,NaN,"2-3,9,12","1,4-8,10-11","1,4,6-11",0,Open water - freshwater,NaN


In [10]:
#Converting column type to string if not already
ground_truth['NOT_SURE'] = ground_truth.NOT_SURE.astype(str)
ground_truth['WATER'] = ground_truth.WATER.astype(str)

In [11]:
cols = ['WATER','NO_WATER','BAD_IMAGE','NOT_SURE']
for col in cols:
    ground_truth[col] = ground_truth[col].str.replace('[','')
    ground_truth[col] = ground_truth[col].str.replace(']','')
    ground_truth[col] = ground_truth[col].str.replace('&','')
    #ground_truth[col] = [''.join(c.split()) for c in ground_truth[col]]

In [12]:
#check whether any nan values in the dataframe and print it out against the column name 
count_nan_in_df = ground_truth.isnull().sum()
print (count_nan_in_df)

PLOT_ID        0
LON            0
LAT            0
FLAGGED        0
ANALYSES       0
S2DATE        55
ENDDATE      119
STARTDATE    119
WATER          0
NO_WATER       0
BAD_IMAGE      0
NOT_SURE       0
CLASS          0
COMMENT      134
dtype: int64


In [13]:
#replacing the name of months with their numerical values
replacements = { 'WATER': {r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'},
               'NO_WATER': {r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'},
               'BAD_IMAGE':{r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'}}

ground_truth.replace(replacements, regex=True, inplace=True)

In [14]:
#ground_truth['S2DATE'] = ground_truth['S2DATE'].str.replace('2019-2019','2018-2018')

In [15]:
def split_str(row, newtable):
#check each row for No-WATER info an update the water column 
    monthstr=row['NO_WATER']
    if monthstr!='0'and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','STARTDATE','ENDDATE','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='0'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow)
#check each row for water info 
    monthstr=row['WATER']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','STARTDATE','ENDDATE','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='1'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow)  # update index / ignore original index
#check each row for bad image 
    monthstr=row['BAD_IMAGE']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','STARTDATE','ENDDATE','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='2'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow) 
    monthstr=row['NOT_SURE']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','STARTDATE','ENDDATE','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='3'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow) 
                
    return newtable

In [16]:
# count_nan_in_df = ground_truth.isnull().sum()
# print (count_nan_in_df)

In [17]:
#ground_truth.dtypes

In [18]:
#Making an empty dataframe
result = pd.DataFrame()

In [19]:
for irow in range(len(ground_truth)):
    result=split_str(ground_truth.iloc[irow], result)
    result.update(result)

In [20]:
result.shape
#result.loc[13]#this shows all the table 

(3537, 16)

In [21]:
result

,ANALYSES,BAD_IMAGE,CLASS,COMMENT,ENDDATE,FLAGGED,LAT,LON,MONTH,NOT_SURE,NO_WATER,PLOT_ID,SENTINEL2YEAR,STARTDATE,WATER,WATERFLAG
0,1.0,6-9,Open water - marine,NaN,NaN,0.0,8.894824,-13.157104,01,0,0,137483360.0,2018,NaN,1-12,1
0,1.0,6-9,Open water - marine,NaN,NaN,0.0,8.894824,-13.157104,02,0,0,137483360.0,2018,NaN,1-12,1
0,1.0,6-9,Open water - marine,NaN,NaN,0.0,8.894824,-13.157104,03,0,0,137483360.0,2018,NaN,1-12,1
0,1.0,6-9,Open water - marine,NaN,NaN,0.0,8.894824,-13.157104,04,0,0,137483360.0,2018,NaN,1-12,1
0,1.0,6-9,Open water - marine,NaN,NaN,0.0,8.894824,-13.157104,05,0,0,137483360.0,2018,NaN,1-12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,1.0,"1,3-10",Open water - freshwater,NaN,NaN,0.0,4.549805,8.870069,06,0,"1,3-10",137483529.0,2018,NaN,"2,11-12",2
171,1.0,"1,3-10",Open water - freshwater,NaN,NaN,0.0,4.549805,8.870069,07,0,"1,3-10",137483529.0,2018,NaN,"2,11-12",2
171,1.0,"1,3-10",Open water - freshwater,NaN,NaN,0.0,4.549805,8.870069,08,0,"1,3-10",137483529.0,2018,NaN,"2,11-12",2
171,1.0,"1,3-10",Open water - freshwater,NaN,NaN,0.0,4.549805,8.870069,09,0,"1,3-10",137483529.0,2018,NaN,"2,11-12",2


In [22]:
# indexNames = result[result.duplicated(['LAT', 'LON','MONTH'], keep=False)]
# indexNames.shape

In [23]:
result = result[['PLOT_ID', 'LON', 'LAT','FLAGGED','ANALYSES','SENTINEL2YEAR', 'STARTDATE','ENDDATE','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS', 'COMMENT', 'MONTH','WATERFLAG']]

In [24]:
indexNames = result[result.duplicated(['LAT', 'LON','MONTH'], keep=False) & (result['WATERFLAG'] <= '1') & (result['NOT_SURE']!='0')].index
result.drop(indexNames , inplace=True)

In [25]:
result

,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2YEAR,STARTDATE,ENDDATE,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG
0,137483360.0,-13.157104,8.894824,0.0,1.0,2018,NaN,NaN,1-12,0,6-9,0,Open water - marine,NaN,01,1
0,137483360.0,-13.157104,8.894824,0.0,1.0,2018,NaN,NaN,1-12,0,6-9,0,Open water - marine,NaN,02,1
0,137483360.0,-13.157104,8.894824,0.0,1.0,2018,NaN,NaN,1-12,0,6-9,0,Open water - marine,NaN,03,1
0,137483360.0,-13.157104,8.894824,0.0,1.0,2018,NaN,NaN,1-12,0,6-9,0,Open water - marine,NaN,04,1
0,137483360.0,-13.157104,8.894824,0.0,1.0,2018,NaN,NaN,1-12,0,6-9,0,Open water - marine,NaN,05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,137483529.0,8.870069,4.549805,0.0,1.0,2018,NaN,NaN,"2,11-12","1,3-10","1,3-10",0,Open water - freshwater,NaN,06,2
171,137483529.0,8.870069,4.549805,0.0,1.0,2018,NaN,NaN,"2,11-12","1,3-10","1,3-10",0,Open water - freshwater,NaN,07,2
171,137483529.0,8.870069,4.549805,0.0,1.0,2018,NaN,NaN,"2,11-12","1,3-10","1,3-10",0,Open water - freshwater,NaN,08,2
171,137483529.0,8.870069,4.549805,0.0,1.0,2018,NaN,NaN,"2,11-12","1,3-10","1,3-10",0,Open water - freshwater,NaN,09,2


In [28]:
#group by PLOT Id and then do the estimation of the row number for each plot number in column Month  
count = result.groupby(['PLOT_ID'])['MONTH'].count()
count.to_csv('../Supplementary_data/Validation/Refined/CEO_4_AFRIGIST_count.csv')

In [26]:
#save the dataframe as csv file 
result.to_csv('../Supplementary_data/Validation/Refined/CEO_4_AFRIGIST_2020-09-15.csv')

In [3]:
#joining dataframes together and extract one csv for each partner institution 
DF = glob.glob('../Supplementary_data/Validation/Refined/*_AFRIGIST_*.csv')
frame = []
for d in DF: 
    f = pd.read_csv(d,delimiter=",")
    frame.append(f)
out = pd.concat(frame)
out

,Unnamed: 0,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2YEAR,STARTDATE,ENDDATE,WATER,...,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG,SENTINEL2Y,geometry,CLASS_WET,CLEAR_OBS
0,0.0,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,...,2,0,Open water - freshwater,NaN,1,1.0,NaN,NaN,NaN,NaN
1,0.0,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,...,2,0,Open water - freshwater,NaN,2,1.0,NaN,NaN,NaN,NaN
2,0.0,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,...,2,0,Open water - freshwater,NaN,3,1.0,NaN,NaN,NaN,NaN
3,0.0,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,...,2,0,Open water - freshwater,NaN,4,1.0,NaN,NaN,NaN,NaN
4,0.0,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,...,2,0,Open water - freshwater,NaN,5,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2657,188.0,137482804.0,13.838962,-9.369115,0.0,1.0,2018.0,1/09/2018,5/09/2018,8,...,"1-7,9-12",0,Open water - freshwater,na,7,2.0,NaN,NaN,NaN,NaN
2658,188.0,137482804.0,13.838962,-9.369115,0.0,1.0,2018.0,1/09/2018,5/09/2018,8,...,"1-7,9-12",0,Open water - freshwater,na,9,2.0,NaN,NaN,NaN,NaN
2659,188.0,137482804.0,13.838962,-9.369115,0.0,1.0,2018.0,1/09/2018,5/09/2018,8,...,"1-7,9-12",0,Open water - freshwater,na,10,2.0,NaN,NaN,NaN,NaN
2660,188.0,137482804.0,13.838962,-9.369115,0.0,1.0,2018.0,1/09/2018,5/09/2018,8,...,"1-7,9-12",0,Open water - freshwater,na,11,2.0,NaN,NaN,NaN,NaN


In [4]:
out.columns

Index(['Unnamed: 0', 'PLOT_ID', 'LON', 'LAT', 'FLAGGED', 'ANALYSES',
       'SENTINEL2YEAR', 'STARTDATE', 'ENDDATE', 'WATER', 'NO_WATER',
       'BAD_IMAGE', 'NOT_SURE', 'CLASS', 'COMMENT', 'MONTH', 'WATERFLAG',
       'SENTINEL2Y', 'geometry', 'CLASS_WET', 'CLEAR_OBS'],
      dtype='object')

In [5]:
out = out.drop(['Unnamed: 0','SENTINEL2Y', 'geometry', 'CLASS_WET', 'CLEAR_OBS'], axis=1)

In [7]:
out

,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2YEAR,STARTDATE,ENDDATE,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG
0,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,0,2,0,Open water - freshwater,NaN,1,1.0
1,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,0,2,0,Open water - freshwater,NaN,2,1.0
2,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,0,2,0,Open water - freshwater,NaN,3,1.0
3,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,0,2,0,Open water - freshwater,NaN,4,1.0
4,137483175.0,30.463813,-26.653807,0.0,1.0,2018.0,NaN,NaN,1-12,0,2,0,Open water - freshwater,NaN,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2657,137482804.0,13.838962,-9.369115,0.0,1.0,2018.0,1/09/2018,5/09/2018,8,"1-7,9-12","1-7,9-12",0,Open water - freshwater,na,7,2.0
2658,137482804.0,13.838962,-9.369115,0.0,1.0,2018.0,1/09/2018,5/09/2018,8,"1-7,9-12","1-7,9-12",0,Open water - freshwater,na,9,2.0
2659,137482804.0,13.838962,-9.369115,0.0,1.0,2018.0,1/09/2018,5/09/2018,8,"1-7,9-12","1-7,9-12",0,Open water - freshwater,na,10,2.0
2660,137482804.0,13.838962,-9.369115,0.0,1.0,2018.0,1/09/2018,5/09/2018,8,"1-7,9-12","1-7,9-12",0,Open water - freshwater,na,11,2.0


In [8]:
out.to_csv('../Supplementary_data/Validation/Refined/CEO_AFRIGIST_2020-09-15.csv')

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** January 2020

**Compatible datacube version:** 

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)